# **Criando o Banco de Dados**

Notebook com os comandos necessários para criar um banco SQL no SQLite e carregar os dados dos arquivos fornecidos nele.

**Arquivos fornecidos:**
- *train_houses.xlsx*
- *test_houses.xlsx*

## Imports

In [1]:
import pandas as pd
import sqlite3

## Criando conexão ao banco SQL

In [3]:
# Conectar ao banco de dados SQLite (ou criar se não existir)
conn = sqlite3.connect('data/house_prices.db')

## Corrigindo formatação dos dados brutos

- Os dados brutos estavam em arquivos xlsx.
- Mas em inspeção visual, eles eram dados separados por vírgulas, como um csv.
- Por algum motivo, ler o XLSX no Pandas e depois gravar em CSV não estava funcionando. O Pandas não reconhecia o delimitador ','.
- Por isso, eu abri os arquivos XLSX no Excel;
- em seguida, eu **localizei e substituí** todas as ocorrências de ', ' (vírgula seguida de espaço), que era o causador da confusão com o Pandas.
- Por fim, **salvei como** CSV UTF-8 esse arquivo XLSX.
- Agora é só ler o arquivo CSV pelo Pandas e gravá-lo no banco SQL. 

In [60]:
# Ler o arquivo CSV corrigido
df_train = pd.read_csv('data/train_houses_corrigido.csv', sep=',', encoding='utf-8')

df_train.head()

,Type,Region,MunicipalityCode,Prefecture,Municipality,DistrictName,NearestStation,TimeToNearestStation,MinTimeToNearestStation,MaxTimeToNearestStation,...,Breadth,CityPlanning,CoverageRatio,FloorAreaRatio,Period,Year,Quarter,Renovation,Remarks,TradePrice
0,Pre-owned Condominiums etc.,NaN,13103.0,Tokyo,Minato Ward,Kaigan,Takeshiba,1,1.0,1.0,...,NaN,Quasi-industrial Zone,60.0,400.0,1st quarter 2011,2011.0,1.0,Done,NaN,24000000.0
1,Residential Land(Land and Building),Residential Area,13120.0,Tokyo,Nerima Ward,Nishiki,Kamiitabashi,15,15.0,15.0,...,4.0,Category I Exclusively Low-story Residential Zone,60.0,200.0,3rd quarter 2013,2013.0,3.0,NaN,Dealings including private road,51000000.0
2,Residential Land(Land Only),Residential Area,13201.0,Tokyo,Hachioji City,Shimoongatamachi,Takao (Tokyo),1H-1H30,60.0,90.0,...,4.5,Category I Exclusively Low-story Residential Zone,40.0,80.0,4th quarter 2007,2007.0,4.0,NaN,NaN,14000000.0
3,Pre-owned Condominiums etc.,NaN,13208.0,Tokyo,Chofu City,Kamiishiwara,Nishichofu,16,16.0,16.0,...,NaN,Quasi-industrial Zone,60.0,200.0,2nd quarter 2015,2015.0,2.0,Not yet,NaN,23000000.0
4,Residential Land(Land Only),Residential Area,13117.0,Tokyo,Kita Ward,Shimo,Shimo,6,6.0,6.0,...,4.5,Category I Exclusively Medium-high Residential...,60.0,200.0,4th quarter 2015,2015.0,4.0,NaN,NaN,33000000.0


## Carregando dados no banco SQL

In [13]:
def process_and_save(df, table_name):
    # Obter os nomes das colunas
    column_names = get_column_names(df)
    
    # Separar os dados e definir os nomes das colunas
    data = df[1:]  # Ignorar a primeira linha que contém os nomes das colunas
    data = data[0].str.split(',', expand=True)  # Separar os valores da linha
    
    # Ajustar o número de colunas se necessário
    if len(data.columns) != len(column_names):
        print(f"Warning: Number of columns in data ({len(data.columns)}) does not match the number of column names ({len(column_names)}).")
        min_length = min(len(data.columns), len(column_names)) # Define o tamanho do dataset = número de colunas
        data = data.iloc[:, :min_length]  # Truncar ou preencher com NaN se necessário
        column_names = column_names[:min_length]  # Ajustar os nomes das colunas
    
    data.columns = column_names  # Definir os nomes das colunas
    
    # Salvar no banco de dados
    data.to_sql(table_name, conn, if_exists='replace', index=False)

In [14]:
# Processar e salvar os DataFrames
process_and_save(df_train, 'train')
process_and_save(df_test, 'test')

In [15]:
# Fechar a conexão com o banco de dados
conn.close()

print("Dados carregados com sucesso no banco de dados SQLite.")

Dados carregados com sucesso no banco de dados SQLite.
